In [27]:
import attr
import os
import ujson
import bz2
import random

import pandas as pd

from glob import glob
from tqdm import tqdm
from multiprocessing import Pool
from itertools import islice
from functools import partial
from boltons.iterutils import pairwise
from collections import Counter
from scipy import stats
from sklearn.model_selection import train_test_split

from IPython.display import display, Markdown

In [2]:
def zip_offset(seq):
    """Yield (item, 0-1 offset).
    """
    size = len(seq)
    for i, item in enumerate(seq):
        offset = i / (size - 1) if (size - 1) else 0
        yield item, offset

In [3]:
def map_segment(func, path):
    """Parse JSON segment, apply worker function.
    """
    results = []
    
    with bz2.open(path) as fh:
        for line in fh:
            results.append(func(ujson.loads(line)))
            
    return results

In [4]:
@attr.s
class Corpus:
    
    root = attr.ib()
    
    def paths(self):
        return glob(os.path.join(self.root, '*.bz2'))
                        
    def map_novels(self, func, shuffle=True):
        """Apply a worker to segment files in parallel.
        """
        paths = self.paths()
        
        if shuffle:
            random.shuffle(paths)
        
        with Pool() as p:
            
            worker = partial(map_segment, func)
            
            for results in p.imap_unordered(worker, paths):
                yield from results

In [5]:
corpus = Corpus('../../data/chicago-bins-tokens.json/')

In [71]:
def _match_suffixes(query, group, o1, o2, novel, width=5):
    """Match suffix windows after a query token.
    """
    results = []
    
    for i, (t, offset) in enumerate(zip_offset(novel['tokens'])):
        if o1 < offset < o2 and t.lower() == query:
            
            suffix = novel['tokens'][i+1:i+1+width]
            
            results.append((novel['identifier'], group, query, suffix))
    
    return results

In [72]:
def match_suffixes(query, group, o1, o2, width=5, skim=None):
    
    worker = partial(_match_suffixes, query, group, o1, o2, width=width)

    results = corpus.map_novels(worker)
    results = tqdm(islice(results, skim))

    return [r for b in results for r in b]

In [79]:
a0 = match_suffixes('a', 0, 0, 0.5, skim=None)

6638it [03:23, 32.69it/s]


In [74]:
a1 = match_suffixes('a', 1, 0.95, 1, skim=100)

100it [00:04, 24.47it/s]


In [75]:
the0 = match_suffixes('the', 0, 0, 0.5, skim=100)

100it [00:06, 15.29it/s]


In [76]:
the1 = match_suffixes('the', 1, 0.95, 1, skim=100)

100it [00:04, 24.29it/s]


In [77]:
rows = a0 + a1 + the0 + the1
df = pd.DataFrame(rows, columns=('chicago_id', 'group', 'token', 'suffix'))

In [78]:
df

,chicago_id,group,token,suffix
0,20511,0,a,"[nightgown, ., It, was, only]"
1,20511,0,a,"[legendary, Greek, figure, sculptured, in]"
2,20511,0,a,"[thousand, times, in, the, movies]"
3,20511,0,a,"[real, -, life, enactment, of]"
4,20511,0,a,"[couple, of, detectives, up, to]"
5,20511,0,a,"[pretty, girl, ,, a, young]"
6,20511,0,a,"[young, girl, in, her, early]"
7,20511,0,a,"[closer, look, at, the, full]"
8,20511,0,a,"[nice, spring, day, ., The]"
9,20511,0,a,"[grip, on, the, rust, -]"
